# Парсинг данных

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import time
import re
import random
import json

import requests
from bs4 import BeautifulSoup
import base64
import lxml

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import pandas as pd

In [ ]:
pd.set_option('max_colwidth', 300)
pd.set_option('display.width', 480)

In [58]:
# Получение HTML-кода страницы
url = "http://citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&pN=2"
response = requests.get(url)
html_content = response.content

# Создание объекта BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# Нахождение таблицы
table = soup.find("table", class_="tbrd")

# Создание списка для данных таблицы
data = []

# Получение заголовков таблицы
headers = table.find_all('td', class_='tht')
headers_text = [header.get_text(strip=True) for header in headers]

# Получение строк таблицы
rows = table.find_all('tr', class_='tbb')

# Извлечение данных из таблицы
data = []
for row in table.find_all('tr'):
    columns = row.find_all('td')
    if len(columns) > 0:
        # Извлечение ссылки на фото
        photo_url = None #columns[0].find('img')['src']
        # Извлечение остальных данных
        other_data = [column.text.strip() for column in columns[1:]]
        data.append([photo_url] + other_data)

# Создание DataFrame
df = pd.DataFrame(data[3:], columns=['photo', 'date_time', 'apartment', 'district', 'address', 'floor_maxfloors', 'total_area', 'living_area', 'kitchen_area', 'note', 'price', 'phone_name', 'agency', 'email'])

ValueError: 14 columns passed, passed data had 13 columns

In [57]:
df

,photo,date_time,apartment,district,address,floor_maxfloors,total_area,living_area,kitchen_area,note,price,phone_name,agency,email
0,None,20.0812:25,Трехкомнатная,Правобережный,ул. Феодосия Воронова 15/1,2/2,79.00,0.00,10.20,"В коттеджном посёлке Александровский сад в центре города продается двухуровневая квартира.Поселок огорожен и имеет зеленую территорию, есть видеонаблюдение. Отличная транспортная развязка, до остановки общественного транспорта 5 минут пешком. Рядом расположены все необходимые объекты инфраструкт...",6250,79068517084 ... Юлия,Оператор недвижимости Ключи,july0708@mail.ru
1,None,21.0815:52,Двухкомнатная,Ленинский,Вокзальная 144/2,3/10,43.20,0.00,6.00,"bПродам 2-к квартиру по цене 1-к!!!! br/bbrЗАДАТОК!!!brПод ремонт, на 3 этаже в 10-ти этажном доме, общей площадью 43.2 квм., по адресу ул. Вокзальная, 144/2brДом кирпичный, очень теплый, соседей не слышноbrКомнаты изолированные, с/у раздельный, площадь кухни 6,8 квм, большая прихожая.brbrРАСПОЛ...",2200,79000636628 ... Наталья Владимировна,Оператор недвижимости Ключи,agentprovokator22@gmail.com
2,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,None,19.0812:25,Трехкомнатная,Орджоникидзевский,Галиуллина 51,1/5,58.40,38.60,6.00,"Тихий, богатый озеленением район города! Удачная транспортная развязка! brКвартира расположена на 1 этаже (достаточно высоко!).brДвор очень ухожен, за этим следит старшая дома! Парковки, детские площадки, новые лавочки,.brХорошие входные двери в подъезд, пластиковые окна, свежий ремонт в подъезд...",3450,79617936284 ... Маргарита Равилевна,АН «Монтера Курорт»,1@1.ru
4,None,22.0812:25,Однокомнатная,Правобережный,Ленина пр-т 91,2/9,35.60,13.00,12.50,Предлагаю Вашему вниманию полноценную 1 комнатную квартиру 35 м2 в девятиэтажном доме в Правобережном районе.br- до остановки 2 мин пешком;br- Продуктовые магазины и аптека в доме;br- Детский сад 6;br- Школа 66;br- Колледж им. Г.И.Носова;br- Университет им. Г.И.Носова;br- Поликлиника в пешей д...,2070,79514654719 ... Константин Сергеевич,АН «Монтера Курорт»,yena.riolux@mail.ru
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,None,21.0811:12,Однокомнатная,Орджоникидзевский,Карла Маркса 227,6/14,38.00,38.00,0.00,"Продам однокомнатную квартиру - студию в идеальном состоянии, заезжай и живи! В квартире сделан качественный ремонт из дорогих материалов. Динамично развивающаяся часть города с хорошо развитой инфраструктурой: магазины, остановки транспорта, новый парк для прогулок, школа, садик - все в шаговой...",3300,+79090974969 ... Буйкли Юлия Анатольевна,,
98,None,09.0716:19,Трехкомнатная ленинградский проект,Ленинский,Калинина 20,2/5,81.00,47.00,8.00,"Трехкомнатная раздельная квартира в Золотом квадрате (Калинина/Ленина),состояние среднее,окна пластиковые,два балкона,один собственник,ЧП,Торг.",4900,8-951-239-99-97 ... Наталья,,
99,None,20.0812:25,Двухкомнатная,Орджоникидзевский,145/1,1/5,49.40,34.00,9.00,"В продаже шикарная 2-х комнатная квартира улучшенной планировки.brbrИнфраструктура: Детский сад 13,Школа 40, Много развивающих центров для детей, ТЦ Казачий,Фитнесцентр Studio fitness, магазин: Магнит, Пятерочка.brbrПланировка: 2 комнаты 12 м2 и 17,5м2, кухня 9 м2. Балкон остеклен пластиком .br...",3600,79954723133 ... Ильсина Александровна,Оператор недвижимости Ключи,Demass80@mail.ru
100,None,22.0812:25,Двухкомнатная,Ленинский,ул. Куйбышева 20,1/3,53.00,36.00,8.00,"b/bСветлая квартира с 2 комнатами 53 кв.м. в Золотом квадрате Ленинского района!brДом расположен в самом красивом районе нашего города! Квартира находится по улице Куйбышева между проспектом Металлургов и ул. Калинина, вдали от шума города. Золотой квадрат Ленинского района отличается разнообраз...",3300,79080564510 ... Даниела Сергеевна,Оператор недвижимости Ключи,daniela.agent@mail.ru


In [25]:
print(page)

None


In [24]:
soup = BeautifulSoup(page.text, 'lxml')
soup

AttributeError: 'NoneType' object has no attribute 'text'

In [62]:
# Загрузка HTML-страницы
url = "http://citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&pN=2"
response = requests.get(url)
html = response.text

In [66]:
# Получение HTML-кода страницы
url = "http://citystar.ru/detal.htm?d=43&nm=%CE%E1%FA%FF%E2%EB%E5%ED%E8%FF+%2D+%CF%F0%EE%E4%E0%EC+%EA%E2%E0%F0%F2%E8%F0%F3+%E2+%E3%2E+%CC%E0%E3%ED%E8%F2%EE%E3%EE%F0%F1%EA%E5&pN=2"
response = requests.get(url)
html_content = response.content

# Создание объекта BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

In [67]:
# Парсинг HTML с помощь BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table')

In [69]:
table

<table border="0" cellpadding="0" cellspacing="0" width="100%">
<tr><td>
<table border="0" cellpadding="0" cellspacing="0" width="100%">
<tr><td align="center"></td></tr>
<tr><td align="center">
<table border="0" cellpadding="0" cellspacing="0">
<tr><td align="right" width="468px"></td>
<td width="20px"></td>
<td align="left" width="468px"></td></tr></table>
</td></tr>
<tr><td align="center"><table border="0" cellpadding="0" cellspacing="2">
<tr><td width="370px"><div style="float:left; padding:10px 20px;"><img alt="ñëàâÿíñêèé" border="0" src="http://www.citystar.ru/content/ñëàâÿíñêèé350-1.gif"/></div></td>
<td width="350px"><div style="float:left; padding:10px 20px;"><a href="brd.asp?bid=1173" target="_blank"><img alt="RekLamA" border="0" src="http://www.citystar.ru/content/Chat_dlya_rieltorov_370kh80.jpg"/></a></div></td>
<td width="370px"></td></tr></table>
</td></tr>
<!--		<tr><td colspan="3"><img src="px.gif" width=1 height=10 alt=""></td></tr>
		<tr valign=top>
			<td align=left>

TypeError: 'NoneType' object is not subscriptable